In [1]:
import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob
import joblib
import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

ROOT = '/kaggle/input/home-credit-credit-risk-model-stability'

from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [2]:
class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df


class Aggregator:
    #Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return expr_max +expr_last+expr_mean
    
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return  expr_max +expr_last+expr_mean
    
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        #expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        return  expr_max +expr_last#+expr_count
    
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols] 
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df

def read_files(regex_path, depth=None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df

def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype(str)
    return df_data, cat_cols

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="object":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")

TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

In [4]:
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        read_file(TRAIN_DIR / "train_applprev_2.parquet", 2),
        read_file(TRAIN_DIR / "train_person_2.parquet", 2)
    ]
}

In [5]:
df_train = feature_eng(**data_store)
print("train data shape:\t", df_train.shape)
del data_store
gc.collect()
df_train = df_train.pipe(Pipeline.filter_cols) 

#-----------------# định nghĩa các cột num_cols và uses sẽ sử dụng (lấy sẵn kết quả của việc xóa bớt các cột có độ tương quan cao - tránh làm việc này gây tràn Ram)
uses=['case_id', 'target', 'WEEK_NUM', 'mean_residualamount_856A', 'mean_dateactivated_425D', 'max_collater_typofvalofguarant_298M', 'last_familystate_726L', 'max_pmtnum_8L', 'max_financialinstitution_382M', 'max_birth_259D', 'max_num_group1_5', 'max_incometype_1044T', 'mindbddpdlast24m_3658935P', 'mean_dateofcredstart_739D', 'max_dpdmaxdateyear_896T', 'mean_annuity_853A', 'max_num_group1_13', 'applicationscnt_867L', 'max_remitter_829L', 'paytype_783L', 'max_annuity_853A', 'max_subjectrole_93M', 'max_childnum_21L', 'last_purposeofcred_874M', 'days30_165L', 'numinstlswithoutdpd_562L', 'opencred_647L', 'last_classificationofcontr_13M', 'max_numberofoutstandinstls_520L', 'last_language1_981M', 'max_pmts_year_507T', 'last_contaddr_smempladdr_334L', 'last_education_927M', 'mean_outstandingdebt_522A', 'last_subjectrole_93M', 'days360_512L', 'deferredmnthsnum_166L', 'totaldebt_9A', 'last_creationdate_885D', 'max_empladdr_district_926M', 'max_numberofcontrsvalue_358L', 'max_debtoverdue_47A', 'max_rejectreason_755M', 'numinstpaidlastcontr_4325080L', 'max_dpdmaxdateyear_596T', 'last_role_1084L', 'pctinstlsallpaidlate1d_3546856L', 'mean_pmts_overdue_1152A', 'max_dtlastpmt_581D', 'mean_refreshdate_3813885D', 'mean_lastupdate_1112D', 'mean_dtlastpmtallstes_3545839D', 'mean_totalamount_996A', 'mean_overdueamount_31A', 'max_overdueamountmaxdateyear_994T', 'last_collaterals_typeofguarante_359M', 'max_conts_type_509L', 'maxdpdlast24m_143P', 'max_cancelreason_3545846M', 'last_processingdate_168D', 'max_pmts_year_1139T', 'numinstpaid_4499208L', 'max_conts_role_79M', 'max_mainoccupationinc_437A', 'mean_credacc_credlmt_575A', 'pctinstlsallpaidearl3d_427L', 'sumoutstandtotal_3546847A', 'max_inittransactioncode_279L', 'dateofbirth_337D', 'mean_monthlyinstlamount_332A', 'last_annuity_853A', 'max_classificationofcontr_13M', 'month_decision', 'last_relationshiptoclient_642T', 'max_mainoccupationinc_384A', 'maxdbddpdtollast12m_3658940P', 'mean_lastupdate_388D', 'last_pmtnum_8L', 'max_description_351M', 'last_sex_738L', 'secondquarter_766L', 'last_credacc_credlmt_575A', 'max_outstandingdebt_522A', 'maxlnamtstart6m_4525199A', 'last_collater_typofvalofguarant_407M', 'fourthquarter_440L', 'price_1097A', 'last_subjectroles_name_541M', 'numnotactivated_1143L', 'mean_dateofrealrepmt_138D', 'last_refreshdate_3813885D', 'last_pmtamount_36A', 'clientscnt_100L', 'max_education_1138M', 'posfpd10lastmonth_333P', 'lastrejectdate_50D', 'mean_downpmt_134A', 'max_dateofcredstart_181D', 'clientscnt_887L', 'numcontrs3months_479L', 'mastercontrelectronic_519L', 'max_nominalrate_281L', 'clientscnt_533L', 'sellerplacecnt_915L', 'mean_overdueamountmax2_398A', 'max_overdueamountmax2date_1142D', 'mean_approvaldate_319D', 'lastdelinqdate_224D', 'currdebt_22A', 'numinsttopaygr_769L', 'mean_dtlastpmt_581D', 'mean_overdueamountmax2_14A', 'avgoutstandbalancel6m_4187114A', 'max_byoccupationinc_3656910L', 'last_purposeofcred_426M', 'max_numberofoverdueinstls_725L', 'currdebtcredtyperange_828A', 'last_firstnonzeroinstldate_307D', 'maritalst_385M', 'last_cancelreason_3545846M', 'days180_256L', 'max_purposeofcred_426M', 'last_collater_typofvalofguarant_298M', 'max_contaddr_matchlist_1032L', 'max_num_group2_13', 'max_downpmt_134A', 'last_isbidproduct_390L', 'last_approvaldate_319D', 'max_num_group1_15', 'max_overdueamountmaxdatemonth_365T', 'clientscnt_157L', 'last_birth_259D', 'clientscnt_1130L', 'numinstlswithdpd10_728L', 'max_sex_738L', 'last_conts_type_509L', 'avgmaxdpdlast9m_3716943P', 'max_collaterals_typeofguarante_669M', 'numinstpaidearly5d_1087L', 'credtype_322L', 'mean_credlmt_935A', 'mean_firstnonzeroinstldate_307D', 'homephncnt_628L', 'max_approvaldate_319D', 'max_numberofinstls_229L', 'datefirstoffer_1144D', 'max_overdueamountmaxdateyear_2T', 'downpmt_116A', 'lastapprdate_640D', 'last_downpmt_134A', 'pmtnum_254L', 'pctinstlsallpaidlate6d_3546844L', 'max_collater_typofvalofguarant_407M', 'lastrejectcommoditycat_161M', 'max_empls_economicalst_849M', 'last_conts_role_79M', 'last_subjectroles_name_838M', 'pmtssum_45A', 'maxdpdinstldate_3546855D', 'disbursementtype_67L', 'max_status_219L', 'mean_totaloutstanddebtvalue_39A', 'last_pmts_month_706T', 'max_currdebt_94A', 'lastactivateddate_801D', 'last_num_group2_13', 'maxdpdfrom6mto36m_3546853P', 'sumoutstandtotalest_4493215A', 'mean_dateofcredend_353D', 'max_pmts_dpd_303P', 'numinstregularpaid_973L', 'last_classificationofcontr_400M', 'max_credamount_590A', 'max_familystate_447L', 'mean_dpdmax_757P', 'last_status_219L', 'mean_mainoccupationinc_437A', 'last_outstandingdebt_522A', 'max_numberofcontrsvalue_258L', 'numinstpaidearly_338L', 'maxdpdlast3m_392P', 'lastst_736L', 'last_rejectreason_755M', 'numinstpaidearly3d_3546850L', 'annuitynextmonth_57A', 'last_incometype_1044T', 'last_num_group1_6', 'last_credamount_590A', 'max_subjectroles_name_838M', 'applications30d_658L', 'max_empl_employedfrom_271D', 'last_cacccardblochreas_147M', 'pmtscount_423L', 'avginstallast24m_3658937A', 'mindbdtollast24m_4525191P', 'posfstqpd30lastmonth_3976962P', 'max_monthlyinstlamount_674A', 'last_postype_4733339M', 'description_5085714M', 'numinstls_657L', 'maxoutstandbalancel12m_4187113A', 'maxdbddpdlast1m_3658939P', 'max_contaddr_smempladdr_334L', 'numinstlswithdpd5_4187116L', 'firstquarter_103L', 'max_dateofrealrepmt_138D', 'maxdpdinstlnum_3546846P', 'max_processingdate_168D', 'last_financialinstitution_591M', 'numpmtchanneldd_318L', 'last_empls_economicalst_849M', 'max_periodicityofpmts_837L', 'amtinstpaidbefduel24m_4187115A', 'totalsettled_863A', 'clientscnt_946L', 'lastapprcredamount_781A', 'last_financialinstitution_382M', 'mean_overdueamountmax2date_1002D', 'numactivecreds_622L', 'mean_numberofoverdueinstlmaxdat_641D', 'mean_monthlyinstlamount_674A', 'last_safeguarantyflag_411L', 'max_relationshiptoclient_642T', 'education_1103M', 'pctinstlsallpaidlate4d_3546849L', 'last_pmts_year_1139T', 'max_residualamount_488A', 'avgpmtlast12m_4525200A', 'mean_creationdate_885D', 'clientscnt_257L', 'max_nominalrate_498L', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'last_empls_employer_name_740M', 'lastapplicationdate_877D', 'max_lastupdate_1112D', 'clientscnt_493L', 'max_debtoutstand_525A', 'dtlastpmtallstes_4499206D', 'last_education_1138M', 'max_numberofinstls_320L', 'last_maxdpdtolerance_577P', 'max_firstnonzeroinstldate_307D', 'lastrejectreasonclient_4145040M', 'max_num_group1', 'responsedate_1012D', 'max_numberofoverdueinstlmaxdat_148D', 'mean_totalamount_6A', 'max_employedfrom_700D', 'max_contractst_964M', 'mean_totaloutstanddebtvalue_668A', 'avgdpdtolclosure24_3658938P', 'maxdbddpdtollast6m_4187119P', 'max_pmts_month_706T', 'cntincpaycont9m_3716944L', 'max_credtype_587L', 'mean_overdueamount_659A', 'cntpmts24_3658933L', 'max_empls_employer_name_740M', 'max_num_group2_15', 'last_currdebt_94A', 'maxdebt4_972A', 'max_empl_employedtotal_800L', 'requesttype_4525192L', 'max_num_group1_9', 'max_collater_valueofguarantee_876L', 'mastercontrexist_109L', 'max_safeguarantyflag_411L', 'max_postype_4733339M', 'lastcancelreason_561M', 'max_collater_valueofguarantee_1124L', 'max_empl_industry_691L', 'mobilephncnt_593L', 'last_persontype_1072L', 'clientscnt_304L', 'numinstregularpaidest_4493210L', 'max_dateactivated_425D', 'maininc_215A', 'max_empladdr_zipcode_114M', 'maxannuity_159A', 'mean_actualdpd_943P', 'mean_instlamount_768A', 'mean_currdebt_94A', 'numrejects9m_859L', 'numactivecredschannel_414L', 'max_numberofoutstandinstls_59L', 'last_subjectrole_182M', 'maxdpdtolerance_374P', 'last_pmts_month_158T', 'maritalst_893M', 'max_numberofoverdueinstlmax_1151L', 'isbidproduct_1095L', 'last_personindex_1023L', 'last_num_group2_14', 'last_mainoccupationinc_384A', 'twobodfilling_608L', 'max_subjectroles_name_541M', 'max_numberofoverdueinstls_834L', 'clientscnt_360L', 'last_empladdr_zipcode_114M', 'max_contractst_545M', 'max_language1_981M', 'max_cacccardblochreas_147M', 'firstdatedue_489D', 'posfpd30lastmonth_3976960P', 'last_contractst_964M', 'lastrejectreason_759M', 'max_overdueamountmaxdatemonth_284T', 'max_dateofcredend_353D', 'credamount_770A', 'disbursedcredamount_1113A', 'max_classificationofcontr_400M', 'last_credtype_587L', 'applicationcnt_361L', 'max_collaterals_typeofguarante_359M', 'last_contaddr_matchlist_1032L', 'max_dpdmaxdatemonth_89T', 'mean_outstandingamount_354A', 'mean_totaldebtoverduevalue_718A', 'max_pmtamount_36A', 'mean_pmts_dpd_303P', 'max_type_25L', 'mean_credamount_590A', 'max_role_1084L', 'max_num_group2_14', 'last_inittransactioncode_279L', 'max_credacc_credlmt_575A', 'responsedate_4527233D', 'mean_pmts_overdue_1140A', 'last_rejectreasonclient_4145042M', 'pctinstlsallpaidlat10d_839L', 'last_dateactivated_425D', 'max_refreshdate_3813885D', 'max_education_927M', 'daysoverduetolerancedd_3976961L', 'last_num_group1', 'max_rejectreasonclient_4145042M', 'max_periodicityofpmts_1102L', 'thirdquarter_1082L', 'max_familystate_726L', 'last_type_25L', 'actualdpdtolerance_344P', 'mean_overdueamountmax_35A', 'mean_pmts_dpd_1073P', 'last_empladdr_district_926M', 'max_isbidproduct_390L', 'mean_credlmt_230A', 'inittransactioncode_186L', 'commnoinclast6m_3546845L', 'applicationscnt_1086L', 'numactiverelcontr_750L', 'max_purposeofcred_874M', 'max_dtlastpmtallstes_3545839D', 'weekday_decision', 'last_pmts_year_507T', 'max_relationshiptoclient_415T', 'numinstpaidearly5dest_4493211L', 'last_actualdpd_943P', 'last_collaterals_typeofguarante_669M', 'max_pmts_month_158T', 'max_financialinstitution_591M', 'last_description_351M', 'clientscnt_1022L', 'last_mainoccupationinc_437A', 'last_dtlastpmtallstes_3545839D', 'eir_270L', 'mean_outstandingamount_362A', 'clientscnt_1071L', 'birthdate_574D', 'education_88M', 'max_subjectrole_182M', 'last_contractst_545M', 'avgdbddpdlast3m_4187120P', 'paytype1st_925L', 'firstclxcampaign_1125D', 'mean_maxdpdtolerance_577P', 'numincomingpmts_3546848L', 'numinstlsallpaid_934L', 'lastapprcommoditycat_1041M', 'numinstpaidlate1d_3546852L', 'maxinstallast24m_3658928A', 'mean_dateofcredend_289D', 'applicationscnt_464L', 'max_totalamount_6A', 'max_numberofoverdueinstlmax_1039L', 'max_dpdmaxdatemonth_442T', 'max_lastupdate_388D', 'datelastunpaid_3546854D', 'last_employedfrom_700D']
print(len(uses))
df_train=df_train[uses]

num_cols = ['secondquarter_766L', 'last_refreshdate_3813885D', 'numinstlswithdpd10_728L', 'mean_dateactivated_425D', 'mean_credamount_590A', 'max_dtlastpmt_581D', 'totaldebt_9A', 'clientscnt_304L', 'max_dateactivated_425D', 'clientscnt_887L', 'numinstregularpaidest_4493210L', 'mean_lastupdate_1112D', 'mean_overdueamountmax2_14A', 'max_overdueamountmax2date_1142D', 'max_debtoutstand_525A', 'max_periodicityofpmts_837L', 'mean_dtlastpmtallstes_3545839D', 'last_pmtnum_8L', 'mean_approvaldate_319D', 'last_credacc_credlmt_575A', 'last_pmts_month_706T', 'pctinstlsallpaidlate1d_3546856L', 'max_overdueamountmaxdateyear_994T', 'max_dtlastpmtallstes_3545839D', 'maxannuity_159A', 'firstclxcampaign_1125D', 'pmtssum_45A', 'max_debtoverdue_47A', 'numinsttopaygr_769L', 'mastercontrelectronic_519L', 'applications30d_658L', 'pmtscount_423L', 'lastactivateddate_801D', 'mindbdtollast24m_4525191P', 'dateofbirth_337D', 'max_byoccupationinc_3656910L', 'max_numberofcontrsvalue_358L', 'max_pmts_dpd_303P', 'firstdatedue_489D', 'mean_creationdate_885D', 'cntincpaycont9m_3716944L', 'mean_monthlyinstlamount_674A', 'responsedate_1012D', 'max_numberofoutstandinstls_59L', 'last_num_group1', 'mean_numberofoverdueinstlmaxdat_641D', 'maxdpdfrom6mto36m_3546853P', 'amtinstpaidbefduel24m_4187115A', 'mean_overdueamountmax2date_1002D', 'disbursedcredamount_1113A', 'price_1097A', 'max_outstandingdebt_522A', 'maxdpdlast3m_392P', 'last_mainoccupationinc_384A', 'max_downpmt_134A', 'max_num_group1_9', 'numpmtchanneldd_318L', 'last_num_group2_14', 'max_dateofcredstart_181D', 'numnotactivated_1143L', 'max_collater_valueofguarantee_876L', 'daysoverduetolerancedd_3976961L', 'max_dateofrealrepmt_138D', 'applicationscnt_867L', 'mean_mainoccupationinc_437A', 'sumoutstandtotal_3546847A', 'max_approvaldate_319D', 'mean_credlmt_935A', 'clientscnt_157L', 'homephncnt_628L', 'max_numberofoverdueinstlmax_1039L', 'lastapprdate_640D', 'numrejects9m_859L', 'sumoutstandtotalest_4493215A', 'max_pmts_month_706T', 'avginstallast24m_3658937A', 'maxdbddpdtollast6m_4187119P', 'max_dpdmaxdatemonth_89T', 'pctinstlsallpaidlat10d_839L', 'fourthquarter_440L', 'max_dpdmaxdateyear_896T', 'pctinstlsallpaidlate6d_3546844L', 'mean_actualdpd_943P', 'birthdate_574D', 'max_credamount_590A', 'thirdquarter_1082L', 'mean_pmts_dpd_303P', 'last_persontype_1072L', 'firstquarter_103L', 'maxdbddpdlast1m_3658939P', 'maxdbddpdtollast12m_3658940P', 'mean_dtlastpmt_581D', 'last_dtlastpmtallstes_3545839D', 'weekday_decision', 'avgdpdtolclosure24_3658938P', 'commnoinclast6m_3546845L', 'max_num_group1', 'last_pmts_month_158T', 'annuitynextmonth_57A', 'max_overdueamountmaxdatemonth_284T', 'maxdebt4_972A', 'max_pmts_year_1139T', 'avgdbddpdlast3m_4187120P', 'last_maxdpdtolerance_577P', 'max_residualamount_488A', 'clientscnt_1130L', 'max_lastupdate_1112D', 'mean_outstandingamount_362A', 'max_periodicityofpmts_1102L', 'last_dateactivated_425D', 'max_num_group2_14', 'max_monthlyinstlamount_674A', 'mean_credlmt_230A', 'dtlastpmtallstes_4499206D', 'lastdelinqdate_224D', 'eir_270L', 'downpmt_116A', 'maxdpdlast24m_143P', 'maininc_215A', 'actualdpdtolerance_344P', 'applicationscnt_464L', 'last_creationdate_885D', 'last_outstandingdebt_522A', 'days360_512L', 'max_currdebt_94A', 'last_employedfrom_700D', 'mobilephncnt_593L', 'mean_pmts_overdue_1152A', 'month_decision', 'numinstpaidlate1d_3546852L', 'pmtnum_254L', 'credamount_770A', 'max_numberofinstls_229L', 'clientscnt_100L', 'posfpd10lastmonth_333P', 'mean_totaloutstanddebtvalue_39A', 'max_dpdmaxdatemonth_442T', 'clientscnt_1022L', 'clientscnt_257L', 'mean_totalamount_996A', 'last_birth_259D', 'mean_annuity_853A', 'sellerplacecnt_915L', 'mean_refreshdate_3813885D', 'max_num_group2_15', 'max_dpdmaxdateyear_596T', 'max_num_group1_13', 'lastapprcredamount_781A', 'max_totalamount_6A', 'avgpmtlast12m_4525200A', 'pctinstlsallpaidlate4d_3546849L', 'avgmaxdpdlast9m_3716943P', 'max_annuity_853A', 'numinstlsallpaid_934L', 'deferredmnthsnum_166L', 'mindbddpdlast24m_3658935P', 'clientscnt_360L', 'mean_overdueamount_659A', 'last_num_group1_6', 'clientscnt_533L', 'numinstpaidearly5dest_4493211L', 'numactivecredschannel_414L', 'mean_currdebt_94A', 'numincomingpmts_3546848L', 'max_dateofcredend_353D', 'max_pmts_month_158T', 'cntpmts24_3658933L', 'datefirstoffer_1144D', 'max_numberofoverdueinstls_725L', 'max_credacc_credlmt_575A', 'last_num_group2_13', 'mastercontrexist_109L', 'last_currdebt_94A', 'numinstpaidearly5d_1087L', 'mean_downpmt_134A', 'last_pmts_year_1139T', 'last_processingdate_168D', 'numactivecreds_622L', 'last_actualdpd_943P', 'mean_dpdmax_757P', 'last_credamount_590A', 'last_mainoccupationinc_437A', 'max_num_group2_13', 'max_employedfrom_700D', 'max_numberofoverdueinstlmaxdat_148D', 'mean_firstnonzeroinstldate_307D', 'maxdpdinstlnum_3546846P', 'max_childnum_21L', 'numinstls_657L', 'last_firstnonzeroinstldate_307D', 'posfpd30lastmonth_3976960P', 'totalsettled_863A', 'lastrejectdate_50D', 'max_firstnonzeroinstldate_307D', 'max_overdueamountmaxdatemonth_365T', 'numinstpaidearly3d_3546850L', 'last_approvaldate_319D', 'avgoutstandbalancel6m_4187114A', 'mean_lastupdate_388D', 'max_num_group1_5', 'max_nominalrate_498L', 'mean_monthlyinstlamount_332A', 'clientscnt_946L', 'mean_totaloutstanddebtvalue_668A', 'max_pmts_year_507T', 'max_mainoccupationinc_384A', 'clientscnt_493L', 'max_nominalrate_281L', 'max_collater_valueofguarantee_1124L', 'numinstlswithoutdpd_562L', 'max_pmtamount_36A', 'numinstregularpaid_973L', 'posfstqpd30lastmonth_3976962P', 'max_numberofoverdueinstls_834L', 'max_empl_employedfrom_271D', 'mean_dateofcredstart_739D', 'mean_credacc_credlmt_575A', 'mean_totalamount_6A', 'max_pmtnum_8L', 'lastapplicationdate_877D', 'last_personindex_1023L', 'mean_totaldebtoverduevalue_718A', 'mean_instlamount_768A', 'max_num_group1_15', 'last_pmts_year_507T', 'max_overdueamountmaxdateyear_2T', 'maxinstallast24m_3658928A', 'mean_overdueamountmax_35A', 'currdebtcredtyperange_828A', 'max_numberofoutstandinstls_520L', 'max_lastupdate_388D', 'max_processingdate_168D', 'max_numberofcontrsvalue_258L', 'numactiverelcontr_750L', 'max_refreshdate_3813885D', 'max_numberofinstls_320L', 'max_mainoccupationinc_437A', 'mean_dateofrealrepmt_138D', 'max_birth_259D', 'numinstlswithdpd5_4187116L', 'mean_maxdpdtolerance_577P', 'maxlnamtstart6m_4525199A', 'currdebt_22A', 'datelastunpaid_3546854D', 'maxdpdtolerance_374P', 'mean_pmts_overdue_1140A', 'mean_dateofcredend_289D', 'mean_overdueamountmax2_398A', 'last_downpmt_134A', 'maxoutstandbalancel12m_4187113A', 'last_annuity_853A', 'clientscnt_1071L', 'lastrejectcredamount_222A', 'numinstpaid_4499208L', 'mean_overdueamount_31A', 'numcontrs3months_479L', 'applicationscnt_1086L', 'days30_165L', 'maxdpdinstldate_3546855D', 'applicationcnt_361L', 'numinstpaidearly_338L', 'mean_outstandingdebt_522A', 'max_numberofoverdueinstlmax_1151L', 'last_pmtamount_36A', 'mean_pmts_dpd_1073P', 'numinstpaidlastcontr_4325080L', 'pctinstlsallpaidearl3d_427L', 'responsedate_4527233D', 'days180_256L', 'mean_dateofcredend_353D', 'mean_outstandingamount_354A', 'mean_residualamount_856A']

for col in num_cols:
    df_train = df_train.with_columns(
        pl.col(col).fill_null(-0.1)
    )

encoding_cols = df_train.select(pl.selectors.by_dtype([pl.String, pl.Boolean, pl.Categorical])).columns
print(encoding_cols)
for col in encoding_cols:
    df_train = df_train.with_columns(pl.col(col).fill_null('Missing'))
#-----------------

df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)

train data shape:	 (1526659, 861)
389
['max_collater_typofvalofguarant_298M', 'last_familystate_726L', 'max_financialinstitution_382M', 'max_incometype_1044T', 'max_remitter_829L', 'paytype_783L', 'max_subjectrole_93M', 'last_purposeofcred_874M', 'opencred_647L', 'last_classificationofcontr_13M', 'last_language1_981M', 'last_contaddr_smempladdr_334L', 'last_education_927M', 'last_subjectrole_93M', 'max_empladdr_district_926M', 'max_rejectreason_755M', 'last_role_1084L', 'last_collaterals_typeofguarante_359M', 'max_conts_type_509L', 'max_cancelreason_3545846M', 'max_conts_role_79M', 'max_inittransactioncode_279L', 'max_classificationofcontr_13M', 'last_relationshiptoclient_642T', 'max_description_351M', 'last_sex_738L', 'last_collater_typofvalofguarant_407M', 'last_subjectroles_name_541M', 'max_education_1138M', 'last_purposeofcred_426M', 'maritalst_385M', 'last_cancelreason_3545846M', 'max_purposeofcred_426M', 'last_collater_typofvalofguarant_298M', 'max_contaddr_matchlist_1032L', 'las

In [6]:
sample = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv")
device='gpu'
n_est=6000
k = 100
n_samples = 50000
DRY_RUN = True if sample.shape[0] == 10 else False   
if DRY_RUN:
    device='gpu'
    df_train = df_train.iloc[:50000]
    n_est=600
    k = 2
    n_samples = 5000

print(device)

gpu


In [7]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
        read_file(TEST_DIR / "test_applprev_2.parquet", 2),
        read_file(TEST_DIR / "test_person_2.parquet", 2)
    ]
}

In [8]:
df_test = feature_eng(**data_store)
print("test data shape:\t", df_test.shape)
del data_store
gc.collect()

#-----------------
for col in num_cols:
    df_test = df_test.with_columns(
        pl.col(col).fill_null(-0.1)
    )

for col in encoding_cols:
    df_test = df_test.with_columns(pl.col(col).fill_null('Missing'))
#-----------------

uses.remove("target")
df_test = df_test[uses]
df_test, cat_cols = to_pandas(df_test, cat_cols)
df_test = reduce_mem_usage(df_test)

test data shape:	 (10, 860)
Memory usage of dataframe is 0.03 MB
Memory usage after optimization is: 0.01 MB
Decreased by 50.6%


In [9]:
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)
print("len cat_cols:\t", len(cat_cols))

train data shape:	 (50000, 389)
test data shape:	 (10, 388)
len cat_cols:	 114


**Create more Features**

In [10]:
# Tạo các đặc trưng LOGIC mới từ cột số
df_train['is_high_debt'] = (df_train['totaldebt_9A'] > df_train['mean_totaloutstanddebtvalue_39A']).astype(int)
df_train['is_recently_approved'] = (df_train['max_approvaldate_319D'] > df_train['mean_approvaldate_319D']).astype(int)
df_train['is_high_credamount'] = (df_train['mean_credamount_590A'] > df_train['max_credamount_590A']).astype(int)
df_train['is_high_dtlastpmt'] = (df_train['mean_dtlastpmt_581D'] > df_train['max_dtlastpmt_581D']).astype(int)
df_train['is_same_dtlastpmt'] = (df_train['mean_dtlastpmt_581D'] == df_train['max_dtlastpmt_581D']).astype(int)
# Tạo các đặc trưng tỉ lệ
df_train['mean_credamount_per_totaldebt'] = df_train['mean_credamount_590A'] / (df_train['totaldebt_9A'] + 1e-9)
df_train['mean_overdueamountmax2_per_max_overdueamountmax2date'] = df_train['mean_overdueamountmax2_14A'] / (df_train['max_overdueamountmax2date_1142D'] + 1e-9)
df_train['mean_dtlastpmt_per_max_dtlastpmt'] = df_train['mean_dtlastpmt_581D'] / (df_train['max_dtlastpmt_581D'] + 1e-9)
df_train['mean_overdueamount_per_max_overdueamount'] = df_train['mean_overdueamount_659A'] / (df_train['max_overdueamountmax2date_1142D'] + 1e-9)
df_train['mean_monthlyinstlamount_per_max_monthlyinstlamount'] = df_train['mean_monthlyinstlamount_674A'] / (df_train['max_monthlyinstlamount_674A'] + 1e-9)
df_train['mean_credlmt_per_max_credlmt'] = df_train['mean_credlmt_935A'] / (df_train['max_credamount_590A'] + 1e-9)
df_train['mean_totaloutstanddebtvalue_per_max_outstandingdebt'] = df_train['mean_totaloutstanddebtvalue_39A'] / (df_train['max_outstandingdebt_522A'] + 1e-9)
df_train['mean_totalamount_per_max_totalamount'] = df_train['mean_totalamount_996A'] / (df_train['max_totalamount_6A'] + 1e-9)
df_train['mean_instlamount_per_max_monthlyinstlamount'] = df_train['mean_instlamount_768A'] / (df_train['max_monthlyinstlamount_674A'] + 1e-9)
df_train['mean_dtlastpmtallstes_per_max_dtlastpmtallstes'] = df_train['mean_dtlastpmtallstes_3545839D'] / (df_train['max_dtlastpmtallstes_3545839D'] + 1e-9)
df_train['mean_residualamount_per_max_residualamount'] = df_train['mean_residualamount_856A'] / (df_train['max_residualamount_488A'] + 1e-9)
df_train['mean_mainoccupationinc_per_max_mainoccupationinc'] = df_train['mean_mainoccupationinc_437A'] / (df_train['max_mainoccupationinc_437A'] + 1e-9)
df_train['mean_overdueamountmax2_per_max_overdueamountmax2date'] = df_train['mean_overdueamountmax2_14A'] / (df_train['max_overdueamountmax2date_1142D'] + 1e-9)
#---
df_train['mean_overdueamountmax2_per_mean_overdueamountmax2date'] = df_train['mean_overdueamountmax2_14A'] / (df_train['mean_overdueamountmax2date_1002D'] + 1e-9)
df_train['max_dtlastpmt_per_max_dtlastpmtallstes'] = df_train['max_dtlastpmt_581D'] / (df_train['max_dtlastpmtallstes_3545839D'] + 1e-9)
df_train['mean_actualdpd_per_maxdpdlast3m'] = df_train['mean_actualdpd_943P'] / (df_train['maxdpdlast3m_392P'] + 1e-9)
df_train['max_byoccupationinc_per_mean_mainoccupationinc'] = df_train['max_byoccupationinc_3656910L'] / (df_train['mean_mainoccupationinc_437A'] + 1e-9)
df_train['mean_creationdate_per_last_creationdate'] = df_train['mean_creationdate_885D'] / (df_train['last_creationdate_885D'] + 1e-9)
df_train['max_dateactivated_per_mean_dateactivated'] = df_train['max_dateactivated_425D'] / (df_train['mean_dateactivated_425D'] + 1e-9)
df_train['mean_pmts_overdue_per_mean_overdueamount'] = df_train['mean_pmts_overdue_1152A'] / (df_train['mean_overdueamount_659A'] + 1e-9)
df_train['mean_approvaldate_per_max_approvaldate'] = df_train['mean_approvaldate_319D'] / (df_train['max_approvaldate_319D'] + 1e-9)
df_train['max_periodicityofpmts_per_mean_periodicityofpmts'] = df_train['max_periodicityofpmts_837L'] / (df_train['max_periodicityofpmts_1102L'] + 1e-9)
df_train['mean_lastupdate_per_max_lastupdate'] = df_train['mean_lastupdate_1112D'] / (df_train['max_lastupdate_1112D'] + 1e-9)
# Tạo các đặc trưng mới bằng phép cộng
df_train['total_debt_and_max_debtoutstand'] = df_train['totaldebt_9A'] + df_train['max_debtoutstand_525A']
df_train['mean_and_max_credamount'] = df_train['mean_credamount_590A'] + df_train['max_credamount_590A']
df_train['mean_and_max_dtlastpmt'] = df_train['mean_dtlastpmt_581D'] + df_train['max_dtlastpmt_581D']
df_train['mean_and_max_overdueamountmax2'] = df_train['mean_overdueamountmax2_14A'] + df_train['max_overdueamountmax2date_1142D']
df_train['mean_and_max_residualamount'] = df_train['mean_residualamount_856A'] + df_train['max_residualamount_488A']
df_train['mean_and_max_mainoccupationinc'] = df_train['mean_mainoccupationinc_437A'] + df_train['max_mainoccupationinc_437A']
df_train['mean_and_max_monthlyinstlamount'] = df_train['mean_monthlyinstlamount_674A'] + df_train['max_monthlyinstlamount_674A']
df_train['mean_and_max_overdueamount'] = df_train['mean_overdueamount_659A'] + df_train['max_overdueamountmax2date_1142D']

# Tạo các đặc trưng log
df_train['log_mean_credamount'] = np.log1p(df_train['mean_credamount_590A'])
df_train['log_totaldebt'] = np.log1p(df_train['totaldebt_9A'])
df_train['log_mean_overdueamount'] = np.log1p(df_train['mean_overdueamount_659A'])
df_train['log_mean_monthlyinstlamount'] = np.log1p(df_train['mean_monthlyinstlamount_674A'])
df_train['log_max_dtlastpmt'] = np.log1p(df_train['max_dtlastpmt_581D'])
df_train['log_max_overdueamountmax2date'] = np.log1p(df_train['max_overdueamountmax2date_1142D'])

#--xoathu
df_train['mean_and_max_lastupdate'] = df_train['mean_lastupdate_1112D'] + df_train['max_lastupdate_1112D']
df_train['mean_and_max_approvaldate'] = df_train['mean_approvaldate_319D'] + df_train['max_approvaldate_319D']
df_train['mean_and_max_dateactivated'] = df_train['mean_dateactivated_425D'] + df_train['max_dateactivated_425D']
df_train['mean_and_max_creationdate'] = df_train['mean_creationdate_885D'] + df_train['last_creationdate_885D']
df_train['mean_and_max_dtlastpmtallstes'] = df_train['mean_dtlastpmtallstes_3545839D'] + df_train['max_dtlastpmtallstes_3545839D']
df_train['mean_and_max_overdueamountmax2date'] = df_train['mean_overdueamountmax2date_1002D'] + df_train['max_overdueamountmax2date_1142D']
df_train['mean_and_max_periodicityofpmts'] = df_train['max_periodicityofpmts_837L'] + df_train['max_periodicityofpmts_1102L']
df_train['mean_and_max_actualdpd'] = df_train['mean_actualdpd_943P'] + df_train['maxdpdlast3m_392P']
df_train['log_mean_dtlastpmt'] = np.log1p(df_train['mean_dtlastpmt_581D'])
df_train['log_mean_overdueamountmax2'] = np.log1p(df_train['mean_overdueamountmax2_14A'])
df_train['log_mean_residualamount'] = np.log1p(df_train['mean_residualamount_856A'])
df_train['log_max_residualamount'] = np.log1p(df_train['max_residualamount_488A'])
df_train['log_max_dateactivated'] = np.log1p(df_train['max_dateactivated_425D'])
df_train['log_mean_creationdate'] = np.log1p(df_train['mean_creationdate_885D'])
df_train['log_max_creationdate'] = np.log1p(df_train['last_creationdate_885D'])
df_train['log_mean_dtlastpmtallstes'] = np.log1p(df_train['mean_dtlastpmtallstes_3545839D'])
# Tạo các đặc trưng mới bằng cách tổng hợp các cột số dựa trên các cột danh mục
df_train['mean_credamount_by_incometype'] = df_train.groupby('max_incometype_1044T')['mean_credamount_590A'].transform('mean')
df_train['totaldebt_by_familystate'] = df_train.groupby('max_familystate_726L')['totaldebt_9A'].transform('sum')
df_train['mean_overdueamount_by_relationship'] = df_train.groupby('max_relationshiptoclient_642T')['mean_overdueamount_659A'].transform('mean')
df_train['mean_dtlastpmt_by_status'] = df_train.groupby('max_status_219L')['mean_dtlastpmt_581D'].transform('mean')
#--
df_train['totaldebt_by_last_relationship'] = df_train.groupby('last_relationshiptoclient_642T')['totaldebt_9A'].transform('sum')
df_train['mean_credamount_by_requesttype'] = df_train.groupby('requesttype_4525192L')['mean_credamount_590A'].transform('mean')
df_train['mean_overdueamount_by_commoditycat'] = df_train.groupby('lastapprcommoditycat_1041M')['mean_overdueamount_659A'].transform('mean')
df_train['mean_dtlastpmt_by_education'] = df_train.groupby('education_1103M')['mean_dtlastpmt_581D'].transform('mean')
df_train['mean_credamount_by_maritalst'] = df_train.groupby('maritalst_385M')['mean_credamount_590A'].transform('mean')

#--xoathu
df_train['mean_overdueamount_by_lastrejectreason'] = df_train.groupby('lastrejectreason_759M')['mean_overdueamount_659A'].transform('mean')
df_train['mean_credamount_by_inittransactioncode'] = df_train.groupby('inittransactioncode_186L')['mean_credamount_590A'].transform('mean')
df_train['totaldebt_by_lastcancelreason'] = df_train.groupby('lastcancelreason_561M')['totaldebt_9A'].transform('sum')
df_train['mean_overdueamount_by_postype'] = df_train.groupby('max_postype_4733339M')['mean_overdueamount_659A'].transform('mean')
df_train['mean_dtlastpmt_by_last_status'] = df_train.groupby('last_status_219L')['mean_dtlastpmt_581D'].transform('mean')
df_train['mean_credamount_by_sex'] = df_train.groupby('max_sex_738L')['mean_credamount_590A'].transform('mean')
df_train['totaldebt_by_financialinstitution'] = df_train.groupby('max_financialinstitution_591M')['totaldebt_9A'].transform('sum')

In [11]:
# Tạo các đặc trưng LOGIC mới từ cột số
df_test['is_high_debt'] = (df_test['totaldebt_9A'] > df_test['mean_totaloutstanddebtvalue_39A']).astype(int)
df_test['is_recently_approved'] = (df_test['max_approvaldate_319D'] > df_test['mean_approvaldate_319D']).astype(int)
df_test['is_high_credamount'] = (df_test['mean_credamount_590A'] > df_test['max_credamount_590A']).astype(int)
df_test['is_high_dtlastpmt'] = (df_test['mean_dtlastpmt_581D'] > df_test['max_dtlastpmt_581D']).astype(int)
df_test['is_same_dtlastpmt'] = (df_test['mean_dtlastpmt_581D'] == df_test['max_dtlastpmt_581D']).astype(int)
# Tạo các đặc trưng tỉ lệ
df_test['mean_credamount_per_totaldebt'] = df_test['mean_credamount_590A'] / (df_test['totaldebt_9A'] + 1e-9)
df_test['mean_overdueamountmax2_per_max_overdueamountmax2date'] = df_test['mean_overdueamountmax2_14A'] / (df_test['max_overdueamountmax2date_1142D'] + 1e-9)
df_test['mean_dtlastpmt_per_max_dtlastpmt'] = df_test['mean_dtlastpmt_581D'] / (df_test['max_dtlastpmt_581D'] + 1e-9)
df_test['mean_overdueamount_per_max_overdueamount'] = df_test['mean_overdueamount_659A'] / (df_test['max_overdueamountmax2date_1142D'] + 1e-9)
df_test['mean_monthlyinstlamount_per_max_monthlyinstlamount'] = df_test['mean_monthlyinstlamount_674A'] / (df_test['max_monthlyinstlamount_674A'] + 1e-9)
df_test['mean_credlmt_per_max_credlmt'] = df_test['mean_credlmt_935A'] / (df_test['max_credamount_590A'] + 1e-9)
df_test['mean_totaloutstanddebtvalue_per_max_outstandingdebt'] = df_test['mean_totaloutstanddebtvalue_39A'] / (df_test['max_outstandingdebt_522A'] + 1e-9)
df_test['mean_totalamount_per_max_totalamount'] = df_test['mean_totalamount_996A'] / (df_test['max_totalamount_6A'] + 1e-9)
df_test['mean_instlamount_per_max_monthlyinstlamount'] = df_test['mean_instlamount_768A'] / (df_test['max_monthlyinstlamount_674A'] + 1e-9)
df_test['mean_dtlastpmtallstes_per_max_dtlastpmtallstes'] = df_test['mean_dtlastpmtallstes_3545839D'] / (df_test['max_dtlastpmtallstes_3545839D'] + 1e-9)
df_test['mean_residualamount_per_max_residualamount'] = df_test['mean_residualamount_856A'] / (df_test['max_residualamount_488A'] + 1e-9)
df_test['mean_mainoccupationinc_per_max_mainoccupationinc'] = df_test['mean_mainoccupationinc_437A'] / (df_test['max_mainoccupationinc_437A'] + 1e-9)
df_test['mean_overdueamountmax2_per_max_overdueamountmax2date'] = df_test['mean_overdueamountmax2_14A'] / (df_test['max_overdueamountmax2date_1142D'] + 1e-9)
#---
df_test['mean_overdueamountmax2_per_mean_overdueamountmax2date'] = df_test['mean_overdueamountmax2_14A'] / (df_test['mean_overdueamountmax2date_1002D'] + 1e-9)
df_test['max_dtlastpmt_per_max_dtlastpmtallstes'] = df_test['max_dtlastpmt_581D'] / (df_test['max_dtlastpmtallstes_3545839D'] + 1e-9)
df_test['mean_actualdpd_per_maxdpdlast3m'] = df_test['mean_actualdpd_943P'] / (df_test['maxdpdlast3m_392P'] + 1e-9)
df_test['max_byoccupationinc_per_mean_mainoccupationinc'] = df_test['max_byoccupationinc_3656910L'] / (df_test['mean_mainoccupationinc_437A'] + 1e-9)
df_test['mean_creationdate_per_last_creationdate'] = df_test['mean_creationdate_885D'] / (df_test['last_creationdate_885D'] + 1e-9)
df_test['max_dateactivated_per_mean_dateactivated'] = df_test['max_dateactivated_425D'] / (df_test['mean_dateactivated_425D'] + 1e-9)
df_test['mean_pmts_overdue_per_mean_overdueamount'] = df_test['mean_pmts_overdue_1152A'] / (df_test['mean_overdueamount_659A'] + 1e-9)
df_test['mean_approvaldate_per_max_approvaldate'] = df_test['mean_approvaldate_319D'] / (df_test['max_approvaldate_319D'] + 1e-9)
df_test['max_periodicityofpmts_per_mean_periodicityofpmts'] = df_test['max_periodicityofpmts_837L'] / (df_test['max_periodicityofpmts_1102L'] + 1e-9)
df_test['mean_lastupdate_per_max_lastupdate'] = df_test['mean_lastupdate_1112D'] / (df_test['max_lastupdate_1112D'] + 1e-9)
# Tạo các đặc trưng mới bằng phép cộng
df_test['total_debt_and_max_debtoutstand'] = df_test['totaldebt_9A'] + df_test['max_debtoutstand_525A']
df_test['mean_and_max_credamount'] = df_test['mean_credamount_590A'] + df_test['max_credamount_590A']
df_test['mean_and_max_dtlastpmt'] = df_test['mean_dtlastpmt_581D'] + df_test['max_dtlastpmt_581D']
df_test['mean_and_max_overdueamountmax2'] = df_test['mean_overdueamountmax2_14A'] + df_test['max_overdueamountmax2date_1142D']
df_test['mean_and_max_residualamount'] = df_test['mean_residualamount_856A'] + df_test['max_residualamount_488A']
df_test['mean_and_max_mainoccupationinc'] = df_test['mean_mainoccupationinc_437A'] + df_test['max_mainoccupationinc_437A']
df_test['mean_and_max_monthlyinstlamount'] = df_test['mean_monthlyinstlamount_674A'] + df_test['max_monthlyinstlamount_674A']
df_test['mean_and_max_overdueamount'] = df_test['mean_overdueamount_659A'] + df_test['max_overdueamountmax2date_1142D']

# Tạo các đặc trưng log
df_test['log_mean_credamount'] = np.log1p(df_test['mean_credamount_590A'])
df_test['log_totaldebt'] = np.log1p(df_test['totaldebt_9A'])
df_test['log_mean_overdueamount'] = np.log1p(df_test['mean_overdueamount_659A'])
df_test['log_mean_monthlyinstlamount'] = np.log1p(df_test['mean_monthlyinstlamount_674A'])
df_test['log_max_dtlastpmt'] = np.log1p(df_test['max_dtlastpmt_581D'])
df_test['log_max_overdueamountmax2date'] = np.log1p(df_test['max_overdueamountmax2date_1142D'])

#--xoathu
df_test['mean_and_max_lastupdate'] = df_test['mean_lastupdate_1112D'] + df_test['max_lastupdate_1112D']
df_test['mean_and_max_approvaldate'] = df_test['mean_approvaldate_319D'] + df_test['max_approvaldate_319D']
df_test['mean_and_max_dateactivated'] = df_test['mean_dateactivated_425D'] + df_test['max_dateactivated_425D']
df_test['mean_and_max_creationdate'] = df_test['mean_creationdate_885D'] + df_test['last_creationdate_885D']
df_test['mean_and_max_dtlastpmtallstes'] = df_test['mean_dtlastpmtallstes_3545839D'] + df_test['max_dtlastpmtallstes_3545839D']
df_test['mean_and_max_overdueamountmax2date'] = df_test['mean_overdueamountmax2date_1002D'] + df_test['max_overdueamountmax2date_1142D']
df_test['mean_and_max_periodicityofpmts'] = df_test['max_periodicityofpmts_837L'] + df_test['max_periodicityofpmts_1102L']
df_test['mean_and_max_actualdpd'] = df_test['mean_actualdpd_943P'] + df_test['maxdpdlast3m_392P']
df_test['log_mean_dtlastpmt'] = np.log1p(df_test['mean_dtlastpmt_581D'])
df_test['log_mean_overdueamountmax2'] = np.log1p(df_test['mean_overdueamountmax2_14A'])
df_test['log_mean_residualamount'] = np.log1p(df_test['mean_residualamount_856A'])
df_test['log_max_residualamount'] = np.log1p(df_test['max_residualamount_488A'])
df_test['log_max_dateactivated'] = np.log1p(df_test['max_dateactivated_425D'])
df_test['log_mean_creationdate'] = np.log1p(df_test['mean_creationdate_885D'])
df_test['log_max_creationdate'] = np.log1p(df_test['last_creationdate_885D'])
df_test['log_mean_dtlastpmtallstes'] = np.log1p(df_test['mean_dtlastpmtallstes_3545839D'])
# Tạo các đặc trưng mới bằng cách tổng hợp các cột số dựa trên các cột danh mục
df_test['mean_credamount_by_incometype'] = df_test.groupby('max_incometype_1044T')['mean_credamount_590A'].transform('mean')
df_test['totaldebt_by_familystate'] = df_test.groupby('max_familystate_726L')['totaldebt_9A'].transform('sum')
df_test['mean_overdueamount_by_relationship'] = df_test.groupby('max_relationshiptoclient_642T')['mean_overdueamount_659A'].transform('mean')
df_test['mean_dtlastpmt_by_status'] = df_test.groupby('max_status_219L')['mean_dtlastpmt_581D'].transform('mean')
#--
df_test['totaldebt_by_last_relationship'] = df_test.groupby('last_relationshiptoclient_642T')['totaldebt_9A'].transform('sum')
df_test['mean_credamount_by_requesttype'] = df_test.groupby('requesttype_4525192L')['mean_credamount_590A'].transform('mean')
df_test['mean_overdueamount_by_commoditycat'] = df_test.groupby('lastapprcommoditycat_1041M')['mean_overdueamount_659A'].transform('mean')
df_test['mean_dtlastpmt_by_education'] = df_test.groupby('education_1103M')['mean_dtlastpmt_581D'].transform('mean')
df_test['mean_credamount_by_maritalst'] = df_test.groupby('maritalst_385M')['mean_credamount_590A'].transform('mean')

#--xoathu
df_test['mean_overdueamount_by_lastrejectreason'] = df_test.groupby('lastrejectreason_759M')['mean_overdueamount_659A'].transform('mean')
df_test['mean_credamount_by_inittransactioncode'] = df_test.groupby('inittransactioncode_186L')['mean_credamount_590A'].transform('mean')
df_test['totaldebt_by_lastcancelreason'] = df_test.groupby('lastcancelreason_561M')['totaldebt_9A'].transform('sum')
df_test['mean_overdueamount_by_postype'] = df_test.groupby('max_postype_4733339M')['mean_overdueamount_659A'].transform('mean')
df_test['mean_dtlastpmt_by_last_status'] = df_test.groupby('last_status_219L')['mean_dtlastpmt_581D'].transform('mean')
df_test['mean_credamount_by_sex'] = df_test.groupby('max_sex_738L')['mean_credamount_590A'].transform('mean')
df_test['totaldebt_by_financialinstitution'] = df_test.groupby('max_financialinstitution_591M')['totaldebt_9A'].transform('sum')

In [12]:
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

train data shape:	 (50000, 462)
test data shape:	 (10, 461)


**PCA**

In [13]:
from sklearn.decomposition import PCA
pca = joblib.load('/kaggle/input/pipe1final/pca21.joblib')
pca_columns = [f'PCA_int_{i+1}' for i in range(21)]

In [14]:
principal_components_train = pca.transform(df_train[num_cols])
df_pca_train = pd.DataFrame(principal_components_train, columns=pca_columns, index=df_train.index)
df_train = pd.concat([df_train, df_pca_train], axis=1)

In [15]:
principal_components_test = pca.transform(df_test[num_cols])
df_pca_test = pd.DataFrame(principal_components_test, columns=pca_columns, index=df_test.index)
df_test = pd.concat([df_test, df_pca_test], axis=1)

In [16]:
del df_pca_train, df_pca_test, principal_components_train, principal_components_test
gc.collect()

0

In [17]:
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

train data shape:	 (50000, 483)
test data shape:	 (10, 482)


# -------------------------------------------------------------------------------------------

### Training

In [18]:
params1 = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 10,  
    "learning_rate": 0.05,
    "n_estimators": 2000,  
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 5123,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':64,
    'categorical_feature ': 'auto',
    "device": 'gpu', 
    "verbose": -1,
    "max_bin":245,

}

params2 = {
    "boosting_type": "gbdt",
    "colsample_bynode": 0.8,
    "colsample_bytree": 0.8,
    "device": 'gpu', 
    "extra_trees": True,
    "learning_rate": 0.03,
    "l1_regularization": 0.1,
    "l2_regularization": 10,
    "max_depth": 16,
    "metric": "auc",
    "n_estimators": 2000,
    "num_leaves": 72,
    "objective": "binary",
    "random_state": 12461,
    "verbose": -1,
    "max_bin":245,
}

In [19]:
y = df_train["target"]
weeks = df_train["WEEK_NUM"]
X= df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

In [20]:
from catboost import CatBoostClassifier, Pool

fitted_models_cat = []
fitted_models_lgb = []

iter_cnt = 0


for idx_train, idx_valid in cv.split(X, y, groups=weeks):#
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]# 
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    #------------------------------------------------------
    X_train[cat_cols] = X_train[cat_cols].astype('category')
    X_valid[cat_cols] = X_valid[cat_cols].astype('category')
    
    if iter_cnt % 2 == 0:
        model = lgb.LGBMClassifier(**params1)
    else:
        model = lgb.LGBMClassifier(**params2)
        
    model.fit(
        X_train, y_train,
        eval_set = [(X_valid, y_valid)],
        callbacks = [lgb.log_evaluation(200),lgb.early_stopping(150)] )
    
    fitted_models_lgb.append(model)

    iter_cnt += 1
    
# for i in range(5):
#     cat_model = joblib.load(f'/kaggle/input/pipe1final/cat_{i}.joblib')
#     fitted_models_cat.append(cat_model)

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.737982
Early stopping, best iteration is:
[185]	valid_0's auc: 0.738485
Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.747468
Early stopping, best iteration is:
[149]	valid_0's auc: 0.749703
Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.788682
Early stopping, best iteration is:
[150]	valid_0's auc: 0.790069
Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.743719
Early stopping, best iteration is:
[199]	valid_0's auc: 0.743898
Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.73938
Early stopping, best iteration is:
[106]	valid_0's auc: 0.741553


In [23]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        print(len(estimators))
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        X[cat_cols] = X[cat_cols].astype('category')
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

model = VotingModel(fitted_models_lgb)
model

VotingModel(estimators=[LGBMClassifier(categorical_feature ='auto',
                                       colsample_bynode=0.8,
                                       colsample_bytree=0.8, device='gpu',
                                       extra_trees=True, learning_rate=0.05,
                                       max_bin=245, max_depth=10, metric='auc',
                                       n_estimators=2000, num_leaves=64,
                                       objective='binary', random_state=5123,
                                       reg_alpha=0.1, reg_lambda=10,
                                       verbose=-1),
                        LGBMClassifier(colsample_bynode=0.8,
                                       colsample_...
                                       n_estimators=2000, num_leaves=72,
                                       objective='binary', random_state=12461,
                                       verbose=-1),
                        LGBMClassifier(categorical_feature ='auto',
                                       colsample_bynode=0.8,
                                       colsample_bytree=0.8, device='gpu',
                                       extra_trees=True, learning_rate=0.05,
                                       max_bin=245, max_depth=10, metric='auc',
                                       n_estimators=2000, num_leaves=64,
                                       objective='binary', random_state=5123,
                                       reg_alpha=0.1, reg_lambda=10,
                                       verbose=-1)])

### Submission

In [24]:
df_test = df_test.drop(columns=["WEEK_NUM"])
df_test = df_test.set_index("case_id")


y_pred = pd.Series(model.predict_proba(df_test)[:, 1], index=df_test.index)
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred
df_subm.to_csv("sub1.csv")
df_subm

,score
case_id,
57543,0.016696
57549,0.032931
57551,0.011741
57552,0.051684
57569,0.048434
57630,0.035832
57631,0.077984
57632,0.045347
57633,0.029271
